In [2]:
pip install -U scikit-learn


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
import os

In [5]:
#import os
print(os.listdir("/gdrive/My Drive/Colab Notebooks/Pneumonia"))

['X-Ray']


In [6]:
DATADIR = '/gdrive/My Drive/Colab Notebooks/Pneumonia/X-Ray'

CATEGORIES = CATEGORIES = ["Bacterial","Viral"]

In [7]:
IMG_SIZE = 100

In [9]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                pass
create_all_data()

In [10]:
print("Toplam Görüntü Sayısı: ", len(all_data))

Toplam Görüntü Sayısı:  16167


In [12]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [14]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [15]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(9457, 100, 100, 3)
(5659, 100, 100, 3)
(1051, 100, 100, 3)
(9457,)
(5659,)
(1051,)


In [16]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [17]:
!pip install keras_tuner

In [18]:
from kerastuner.tuners import RandomSearch
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout

model = Sequential()

def build_model(hp):
    model = Sequential()
    model.add(Conv2D(hp.Int('conv1_units', min_value=32, max_value=128, step=32), (3,3), padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(hp.Int('conv2_units', min_value=32, max_value=128, step=32), (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Flatten())
    model.add(Dense(hp.Int('dense1_units', min_value=256, max_value=512, step=256)))
    model.add(Activation('relu'))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Reduce the number of trials
    directory='my_tuner_dir',
    project_name='pneumonia_tuning'
)

tuner.search(x_train, train_yCl, validation_data=(x_val, valid_yCl))

best_model = tuner.get_best_models(num_models=1)[0]
best_model.fit(x_train, train_yCl, validation_data=(x_val, valid_yCl), epochs=25, batch_size=64)


<ipython-input-18-f4495324b67f>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


Reloading Tuner from my_tuner_dir/pneumonia_tuning/tuner0.json
Epoch 1/25
148/148 [==============================] - 30s 112ms/step - loss: 0.4663 - accuracy: 0.7606 - val_loss: 0.4915 - val_accuracy: 0.7146
Epoch 2/25
148/148 [==============================] - 12s 81ms/step - loss: 0.4443 - accuracy: 0.7747 - val_loss: 483.5246 - val_accuracy: 0.3930
Epoch 3/25
148/148 [==============================] - 12s 78ms/step - loss: 0.3958 - accuracy: 0.7924 - val_loss: 220.6620 - val_accuracy: 0.6070
Epoch 4/25
148/148 [==============================] - 12s 79ms/step - loss: 0.3535 - accuracy: 0.8153 - val_loss: 0.4633 - val_accuracy: 0.7631
Epoch 5/25
148/148 [==============================] - 12s 80ms/step - loss: 0.3324 - accuracy: 0.8225 - val_loss: 0.3326 - val_accuracy: 0.8268
Epoch 6/25
148/148 [==============================] - 12s 81ms/step - loss: 0.3189 - accuracy: 0.8328 - val_loss: 0.3952 - val_accuracy: 0.8088
Epoch 7/25
148/148 [==============================] - 12s 82ms/step 

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Test verisi üzerinde modelin tahminlerini al
y_pred = best_model.predict(x_test)

# Tahminlerden sınıf indekslerini elde et
y_pred_classes = np.argmax(y_pred, axis=1)

# Gerçek sınıfları elde et
y_true = np.argmax(test_yCl, axis=1)

# Accuracy hesapla
accuracy = accuracy_score(y_true, y_pred_classes)
print(f'Test Accuracy: {accuracy}')

# Precision hesapla
precision = precision_score(y_true, y_pred_classes, average='weighted')
print(f'Precision: {precision}')

# Recall hesapla
recall = recall_score(y_true, y_pred_classes, average='weighted')
print(f'Recall: {recall}')

# F1 score hesapla
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'F1 Score: {f1}')

# Karmaşıklık Matrisi hesapla
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print('Confusion Matrix:')
print(conf_matrix)

# Classification Report hesapla
class_report = classification_report(y_true, y_pred_classes)
print('Classification Report:')
print(class_report)


177/177 [==============================] - 2s 7ms/step
Test Accuracy: 0.9581198091535607
Precision: 0.9580625496365789
Recall: 0.9581198091535607
F1 Score: 0.9580476506380837
Confusion Matrix:
[[3411  100]
 [ 137 2011]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      3511
           1       0.95      0.94      0.94      2148

    accuracy                           0.96      5659
   macro avg       0.96      0.95      0.96      5659
weighted avg       0.96      0.96      0.96      5659



In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.applications import VGG16
from keras.optimizers import Adam
import tensorflow as tf

# Verileri düzenleme
IMG_SIZE = 100  # Örnek bir görüntü boyutu, kendi projenizdeki değere göre güncelleyin

x_train_reshaped = x_train.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
x_test_reshaped = x_test.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
x_val_reshaped = x_val.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

# Verileri standartlaştırma
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_reshaped.reshape(x_train_reshaped.shape[0], -1))
x_test_scaled = scaler.transform(x_test_reshaped.reshape(x_test_reshaped.shape[0], -1))
x_val_scaled = scaler.transform(x_val_reshaped.reshape(x_val_reshaped.shape[0], -1))

# Sınıfları kategorik hale getirme
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

# VGG16 modelini yükleme (weights='imagenet' ile önceden eğitilmiş ağırlıkları kullanma)
base_model = VGG16(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet')

# VGG16'nın evrişimli katmanlarını dondurma
for layer in base_model.layers:
    layer.trainable = False

# Yeni bir model oluşturma
model_transfer = Sequential()

# VGG16'ın evrişimli katmanlarını ekleyerek transfer öğrenme
model_transfer.add(base_model)
model_transfer.add(Flatten())
model_transfer.add(Dense(512, activation='relu'))
model_transfer.add(Dropout(0.5))
model_transfer.add(Dense(2, activation='softmax'))

# Modeli derleme
model_transfer.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
history_transfer = model_transfer.fit(x_train_reshaped, train_yCl,
                                      batch_size=64,
                                      epochs=25,
                                      validation_data=(x_val_reshaped, valid_yCl))

# Modelin performansını değerlendirme
score_valid_transfer = model_transfer.evaluate(x_val_reshaped, valid_yCl)
print("Validation Accuracy: ", score_valid_transfer[1])

score_test_transfer = model_transfer.evaluate(x_test_reshaped, test_yCl)
print("Test Accuracy: ", score_test_transfer[1])

score_train_transfer = model_transfer.evaluate(x_train_reshaped, train_yCl)
print("Train Accuracy: ", score_train_transfer[1])


Epoch 1/25
148/148 [==============================] - 22s 105ms/step - loss: 2.2763 - accuracy: 0.7187 - val_loss: 0.4875 - val_accuracy: 0.7764
Epoch 2/25
148/148 [==============================] - 11s 75ms/step - loss: 0.4585 - accuracy: 0.7879 - val_loss: 0.4419 - val_accuracy: 0.7992
Epoch 3/25
148/148 [==============================] - 11s 73ms/step - loss: 0.3806 - accuracy: 0.8291 - val_loss: 0.3587 - val_accuracy: 0.8649
Epoch 4/25
148/148 [==============================] - 10s 69ms/step - loss: 0.3491 - accuracy: 0.8465 - val_loss: 0.3021 - val_accuracy: 0.8858
Epoch 5/25
148/148 [==============================] - 11s 71ms/step - loss: 0.3255 - accuracy: 0.8611 - val_loss: 0.3034 - val_accuracy: 0.8953
Epoch 6/25
148/148 [==============================] - 10s 70ms/step - loss: 0.2945 - accuracy: 0.8714 - val_loss: 0.2907 - val_accuracy: 0.8906
Epoch 7/25
148/148 [==============================] - 10s 70ms/step - loss: 0.2709 - accuracy: 0.8782 - val_loss: 0.2712 - val_accuracy

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Test verisi üzerinde tahmin yapma
y_pred_transfer = model_transfer.predict(x_test_reshaped)
y_pred_classes_transfer = np.argmax(y_pred_transfer, axis=1)
y_true_classes_transfer = np.argmax(test_yCl, axis=1)


# Precision, Recall, F1 Score
precision_transfer = precision_score(y_true_classes_transfer, y_pred_classes_transfer, average='weighted')
recall_transfer = recall_score(y_true_classes_transfer, y_pred_classes_transfer, average='weighted')
f1_transfer = f1_score(y_true_classes_transfer, y_pred_classes_transfer, average='weighted')

print("Precision: ", precision_transfer)
print("Recall: ", recall_transfer)
print("F1 Score: ", f1_transfer)

# Karmaşıklık Matrisi
conf_matrix_transfer = confusion_matrix(y_true_classes_transfer, y_pred_classes_transfer)
print("Confusion Matrix:")
print(conf_matrix_transfer)

# Classification Report
class_report_transfer = classification_report(y_true_classes_transfer, y_pred_classes_transfer)
print("Classification Report:")
print(class_report_transfer)


177/177 [==============================] - 6s 33ms/step
Precision:  0.9517021028381976
Recall:  0.9514048418448489
F1 Score:  0.9514959280351978
Confusion Matrix:
[[3352  159]
 [ 116 2032]]
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      3511
           1       0.93      0.95      0.94      2148

    accuracy                           0.95      5659
   macro avg       0.95      0.95      0.95      5659
weighted avg       0.95      0.95      0.95      5659

